# Hora de Código: Enseñando a Aprender
## El pipeline de data science

Supongamos nos encomiendan la tarea de escribir una sequela para el Libro Don Quixote de la Mancha. Debido a nuestra poca experiencia en obras literarias, y la inifinitesimal probabilidad de que hayamos leído la obra en su totalidad, 

Para lograr el objetivo, será necesario dividir la tarea en tres partes

* **Análisis**
* **Modelación**
* **Producción**

In [2]:
import re
import pickle
import requests
import numpy as np
from io import BytesIO
from unidecode import unidecode
from collections import deque
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, LSTM, Dense, BatchNormalization
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam

from collections import Counter

# Análisis
## Primeros Pasos

Con el fin de escribir una sequela, lo primero que realizaremos será entender la primera parte del libro.

**¿De qué manera podemos obtener acceeso a la obra?**  
1. Transcribir el libro a nuestra computadora 
2. Buscar el libro en línea, copiarlo y pegarlo en algún lugar para tener acceso a este
3. Acceder directamente al libro y no tener que copiar y pegar nada (👍)

Para nuestra suerte, la página _[Project Gutemberg](http://www.gutenberg.org)_ ofrece libros gratuitos en línea.

In [4]:
url = "http://www.gutenberg.org/cache/epub/2000/pg2000.txt"
r = requests.get(url)

In [10]:
corpus = r.text
init_book = corpus.find("En un lugar de la Mancha")
end_book = corpus.find("End of Project Gutenberg's")
corpus = corpus[init_book: end_book]
corpus = unidecode(corpus.replace("\r\n", " "))
corpus = re.sub("[^\w\s]", "", corpus).lower()

In [11]:
corpus[:500]

'en un lugar de la mancha de cuyo nombre no quiero acordarme no ha mucho tiempo que vivia un hidalgo de los de lanza en astillero adarga antigua rocin flaco y galgo corredor una olla de algo mas vaca que carnero salpicon las mas noches duelos y quebrantos los sabados lantejas los viernes algun palomino de anadidura los domingos consumian las tres partes de su hacienda el resto della concluian sayo de velarte calzas de velludo para las fiestas con sus pantuflos de lo mesmo y los dias de entreseman'

In [12]:
tokens = text.split()
tokens[:10]

['en', 'un', 'lugar', 'de', 'la', 'mancha', 'de', 'cuyo', 'nombre', 'no']

We now look to make a sequence of characters

In [13]:
word_counter = Counter(tokens)
word_counter.most_common(10)

[('que', 21246),
 ('de', 18031),
 ('y', 17980),
 ('la', 10224),
 ('a', 9718),
 ('el', 9346),
 ('en', 8109),
 ('no', 6275),
 ('se', 5025),
 ('los', 4701)]

In [14]:
elements = []
for w0, w1 in zip(tokens[0:-1], tokens[1:len(tokens)]):
    element = (w0, w1)
    elements.append(element)

In [15]:
bigrams = Counter(elements)
bigrams.most_common(10)

[(('don', 'quijote'), 2149),
 (('de', 'la'), 2052),
 (('lo', 'que'), 1535),
 (('que', 'no'), 1284),
 (('en', 'el'), 1042),
 (('de', 'los'), 965),
 (('que', 'se'), 939),
 (('en', 'la'), 934),
 (('de', 'su'), 900),
 (('a', 'la'), 900)]

Hasta ahora tenemos lo _bigram_ más comúnes, ¿de qué manera podemos conocer los _bigrams_ que empiecen con ciertas palabras?

In [83]:
topv = sorted(filter(lambda w: w[0] == "don", bigrams),
                     key=lambda w: bigrams[w])[:-5:-1]
for v in topv:
    print(v, bigrams[v])

('don', 'quijote') 2149
('don', 'fernando') 131
('don', 'antonio') 62
('don', 'luis') 36


¿De qué manera podríamos calcular la probabilidad de que Cervantes haya escrito `"quijote"` dado que la palabra precedente a esta es `"don"`?

$$
    \mathbb{P}(\texttt{"quijote"} | \texttt{"don"})
$$

In [88]:
topv = sorted(filter(lambda w: w[1] == "quijote", bigrams),
                     key=lambda w: bigrams[w])[:-5:-1]

wfreq = 0
count = 0
for v in topv:
    if v[0] == "don":
        wfreq = bigrams[v]
    count += bigrams[v]
print(wfreq / count)

0.9981421272642824


In [ ]:
elements = []
for ws in zip(tokens[0:-1], tokens[1:len(tokens)]):
    element = (w0, w1)
    elements.append(element)

$n$-gram estaría compuesto de los elementos
$$
    t_k, t_{k + 1}, t_{k+2}
$$

In [62]:
def make_ngrams(tokens, ngram=2):
    ntokens = len(tokens)
    groups = [
        tokens[slice(i, ntokens - ngram + i )]
    for i in range(ngram)]
    grams = [ws for ws in zip(*groups)]
    return grams

In [63]:
for ws in make_ngrams(tokens, ngram=5)[:5]:
    print(ws)

('en', 'un', 'lugar', 'de', 'la')
('un', 'lugar', 'de', 'la', 'mancha')
('lugar', 'de', 'la', 'mancha', 'de')
('de', 'la', 'mancha', 'de', 'cuyo')
('la', 'mancha', 'de', 'cuyo', 'nombre')


In [74]:
g5 = Counter(make_ngrams(tokens, ngram=6))
g5.most_common(10)

[(('el', 'caballero', 'de', 'la', 'triste', 'figura'), 21),
 (('caballero', 'don', 'quijote', 'de', 'la', 'mancha'), 16),
 (('de', 'don', 'quijote', 'de', 'la', 'mancha'), 16),
 (('senor', 'don', 'quijote', 'de', 'la', 'mancha'), 16),
 (('la', 'sin', 'par', 'dulcinea', 'del', 'toboso'), 14),
 (('todos', 'los', 'dias', 'de', 'mi', 'vida'), 14),
 (('dijo', 'a', 'esta', 'sazon', 'don', 'quijote'), 13),
 (('don', 'quijote', 'de', 'la', 'mancha', 'que'), 12),
 (('don', 'quijote', 'de', 'la', 'mancha', 'y'), 11),
 (('en', 'todos', 'los', 'dias', 'de', 'mi'), 10)]

In [92]:
topv = sorted(filter(lambda w: w[0] == "mi" and w[1] == "perro", g5),
                     key=lambda w: g5[w])[:-5:-1]
for v in topv:
    print(v, g5[v])

In [115]:
re.findall("(?:[a-z]+\s){1,5}perros?(?:[a-z]+\s){1,5}", corpus)

['muerta y medio comida de perros y picada de grajos ',
 'entro el cura de la perroquia y tomando a los ',
 'y desatinada en poder destos perros naturales enemigos nuestros maldita ',
 'otros como hacen a los perros cuando en pendencia estan ',
 'sucedio pues que entre los perros que descargo la carga ',
 'guarda en efeto todos cuantos perros topaba aunque fuesen alanos ',
 'el lugar sino ladridos de perros que atronaban los oidos ',
 'le vea yo comido de perros que asi nos trae ',
 'ciguenas el cristel de los perros el vomito y el ',
 'de que color serian los perros que pariese a lo ',
 'por el ladrido de los perros como por el son ',
 'suyos cuando acosado de los perros y seguido de los ',
 'su propio dinero dos famosos perros para guardar el ganado ']

In [121]:
sequences = make_ngrams(tokens, ngram=10)
sequences[0]

('en', 'un', 'lugar', 'de', 'la', 'mancha', 'de', 'cuyo', 'nombre', 'no')

## Modelación

In [123]:
words = sorted(list(set(tokens)))
vocab_size = len(words)
word_ix = {w:i for i, w in enumerate(words)}
sequences_int = [[word_ix[word] for word in seq] for seq in sequences]
sequences_int = np.array(sequences_int)
sequences_int

array([[ 8127, 21133, 13116, ...,  5904, 14679, 14659],
       [21133, 13116,  6023, ..., 14679, 14659, 17320],
       [13116,  6023, 12349, ..., 14659, 17320,   461],
       ...,
       [22035, 20983, 22034, ..., 20498, 19427,  7773],
       [20983, 22034, 11112, ..., 19427,  7773,  1265],
       [22034, 11112,  6023, ...,  7773,  1265, 21227]])

In [125]:
sequences_int.shape

(376471, 10)

In [126]:
X_train, y_train = sequences_int[:,:-1], sequences_int[:, -1:]

X_train = to_categorical(X_train, num_classes=vocab_size)
y_train = to_categorical(y_train, num_classes=vocab_size)

In [127]:
X_input = Input(X_train.shape[1:])
X = LSTM(100, activation="relu", return_sequences=False)(X_input)
X = Dense(vocab_size, activation="softmax")(X)
model = Model(inputs=X_input, outputs=X)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 9, 22147)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               8899200   
_________________________________________________________________
dense (Dense)                (None, 22147)             2236847   
Total params: 11,136,047
Trainable params: 11,136,047
Non-trainable params: 0
_________________________________________________________________


In [ ]:
optimizer = Adam(lr=0.0001)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
model.fit(X_train, y_train, epochs=100)

In [ ]:
"despues de haber vivido su primera aventura don quijote se sentia meroró eo días esegude acorras no en todo en esas a lo hecho y en entandero el de si mierda prosiba"